# Amazon Recommendation System - Lab

## Introduction

Now that you've gotten an introduction to collaborative filtering and recommendation systems, it's time to put your skills to test and build a recommendation system for a real world dataset! For this lab, you'll be using a dataset regarding the book reviews on the Amazon marketplace. While the previous lesson focused on user-based recommendation systems, you'll apply a parallel process for an item-based recommendation system to recommend similar books at the bottom of the product page.

## Objectives

In this lab you will: 

- Use graph-based similarity metrics to create a collaborative filtering recommender system

## Load the Dataset

In [1]:
import pandas as pd
import networkx as nx
G = nx.Graph()

df = pd.read_csv('books_data.edgelist', names=['source', 'target', 'weight'], delimiter=' ')
df.head()

,source,target,weight
0,0827229534,0804215715,0.7
1,0827229534,156101074X,0.5
2,0827229534,0687023955,0.8
3,0827229534,0687074231,0.8
4,0827229534,082721619X,0.7


## Load the Metadata 

Import the metadata available in the file `'books_meta.txt'` (note it is `'\t'` seperated). 

In [3]:
# Your code here
meta = pd.read_csv('books_meta.txt', delimiter='\t')
meta.head()

,Id,ASIN,Title,Categories,Group,SalesRank,TotalReviews,AvgRating,DegreeCentrality,ClusteringCoeff
0,1,0827229534,Patterns of Preaching: A Sermon Sampler,clergi sermon subject religion preach spiritu ...,Book,396585,2,5.0,8,0.80
1,2,0738700797,Candlemas: Feast of Flames,subject witchcraft earth religion spiritu base...,Book,168596,12,4.5,9,0.85
2,3,0486287785,World War II Allied Fighter Planes Trading Cards,general hobbi subject craft home garden book,Book,1270652,1,5.0,0,0.00
3,4,0842328327,Life Application Bible Commentary: 1 and 2 Tim...,spiritu translat commentari christian book gui...,Book,631289,1,4.0,6,0.79
4,5,1577943082,Prayers That Avail Much for Business: Executive,subject religion spiritu busi christian live w...,Book,455160,0,0.0,4,1.00


## Select Books to Test Your Recommender On

Select a small subset of books that you are interested in generating recommendations for. 

In [6]:
# Your code here
worldwar2 = meta[meta.Title.str.contains('World War II')]
worldwar2

,Id,ASIN,Title,Categories,Group,SalesRank,TotalReviews,AvgRating,DegreeCentrality,ClusteringCoeff
2,3,0486287785,World War II Allied Fighter Planes Trading Cards,general hobbi subject craft home garden book,Book,1270652,1,5.0,0,0.00
13,16,0195110382,War at Sea: A Naval History of World War II,general militari naval ii subject war world bo...,Book,631564,10,4.5,3,0.58
651,896,0395913756,Fighting For Honor : Japanese Americans and Wo...,general subject literatur unit asian state wor...,Book,144238,1,4.0,2,0.00
2817,3916,0964586711,PC Patrol Craft of World War II,general militari naval ii subject war world bo...,Book,1137130,3,5.0,3,0.58
4043,5675,0465001637,"Not Like Us: How Europeans Have Loved, Hated a...",general nonfict subject america cultur unit po...,Book,102107,2,5.0,8,0.00
...,...,...,...,...,...,...,...,...,...,...
390866,539670,0891416781,Deadly Sky: The American Combat Airman in Worl...,general militari ii person narrat subject unit...,Book,906910,3,4.5,1,0.00
391907,541123,0785813020,The Battle of the Atlantic: September 1939-May...,general militari naval ii subject war world bo...,Book,367820,2,4.0,2,0.00
392316,541685,1853673935,The German Order of Battle: Infantry in World ...,general militari ii germani subject war world ...,Book,406652,2,4.5,1,0.00
392539,542006,1861262698,German Automatic Weapons of World War II,general militari ii germani subject weapon war...,Book,199193,10,4.5,3,0.58


## Generate Recommendations for a Few Books of Choice

The `'books_data.edgelist'` has conveniently already calculated the distance between items for you. Given this preprocessed data, it's time to employ collaborative filtering to generate recommendations! Generate the top 10 recommendations for each book in the subset you chose. Be sure to print the book name that you are generating recommendations for as well as the name of the books being recommended. 

In [21]:
# Your code here
rec_dict = {}
id_name_dict = dict(zip(meta.ASIN, meta.Title))
for row in worldwar2.index:
    book_id = worldwar2.ASIN[row]
    book_name = id_name_dict[book_id]
    most_similar = df[(df.source == book_id) | (df.target == book_id)].sort_values(by='weight', ascending=False).head(10)
    most_similar['source_name'] = most_similar['source'].map(id_name_dict)
    most_similar['target_name'] = most_similar['target'].map(id_name_dict)
    recommendations = []
    for row in most_similar.index:
        if most_similar.source[row] == book_id:
            recommendations.append((most_similar.target_name[row], most_similar.weight[row]))
        else:
            recommendations.append((most_similar.source_name[row], most_similar.weight[row]))
    rec_dict[book_name] = recommendations
    print('Recommendations for:', book_name)
    for r in recommendations:
        print(r)
    print('\n\n')

Recommendations for: World War II Allied Fighter Planes Trading Cards



Recommendations for: War at Sea: A Naval History of World War II
('How They Won the War in the Pacific: Nimitz and His Admirals', 1.0)
('Touched With Fire: The Land War in the South Pacific', 0.89)
("Midway: The Battle that Doomed Japan, the Japanese Navy's Story", 0.82)



Recommendations for: Fighting For Honor : Japanese Americans and World War II
('Remembering Manzanar: Life in a Japanese Relocation Camp', 0.52)
('Runaway Girl : The Artist Louise Bourgeois (Bccb Blue Ribbon Nonfiction Book Award (Awards))', 0.18)



Recommendations for: PC Patrol Craft of World War II
('Splinter Fleet: The Wooden Subchasers of World War II', 0.9)
('U.S. Destroyers: An Illustrated Design History, Revised Edition', 0.41)
('U.S. Cruisers: An Illustrated Design History', 0.36)



Recommendations for: Not Like Us: How Europeans Have Loved, Hated and Transformed American Culture Since World War II
('Seducing the French: The Dilemma 

Recommendations for: German 20Mm Flak in World War II: 1935-1945 (Schiffer Military/Aviation History)
('German Flak in World War II 1939-1945 (Schiffer Military/Aviation History)', 0.0)



Recommendations for: Darkness Visible: Memoir of a World War II Combat Photographer



Recommendations for: Times of Sorrow & Hope Cmbk: Documenting Everyday Life in Pennsylvania During the Depression and World War II : A Photographic Record
('Chronicles, Vol. 1', 0.11)



Recommendations for: V Is for Victory: The American Home Front During World War II (People's History)
('World War II for Kids: A History with 21 Activities', 0.46)
('After the Dancing Days', 0.29)
('Sadako and the Thousand Paper Cranes', 0.27)



Recommendations for: Commandos From The Sea : The History Of Amphibious Special Warfare In World War II And The Korean War



Recommendations for: Scale Aircraft Drawings: World War II (Scale Aircraft Drawings Vol. 2)
('Flying Models', 0.73)
('The Great Book of Fighters', 0.17)
('Theory of

Recommendations for: Blood, Tears and Folly: An Objective Look at World War II
('Winter : A Novel of a Berlin Family', 0.14)



Recommendations for: Wahoo : The Patrols of America's Most Famous World War II Submarine
('Nothing Friendly in the Vicinity: My Patrols on the Submarine Uss Guardfish During Wwii', 0.84)
("The Bravest Man : The Story of Richard O'Kane and U.S. Submariners in the Pacific War", 0.8)
('Silent Running : My Years on a World War II Attack Submarine', 0.76)
('Silent Running: My Years on a World War II Attack Submarine', 0.76)
('United States Submarine Operations in World War II', 0.5)
('Clear the Bridge! : The War Patrols of the U.S.S. Tang', 0.47)
('Unrestricted Warfare : How a New Breed of Officers Led the Submarine Force to Victory in World War II', 0.44)
('Silent Victory: The U.S. Submarine War Against Japan', 0.42)
('Submarine Diary: The Silent Stalking of Japan', 0.42)
('Thunder Below!: The Uss Barb Revolutionizes Submarine Warfare in World War II', 0.42)



Re

Recommendations for: A Spy in Their Midst : The World War II Struggle of a Japanese-American Hero



Recommendations for: Forgotten Fields of America : World War II Bases and Training, Then and Now - Vol. 2
('Military Aircraft Boneyards', 0.6)
('Warship Boneyards', 0.46)
('Forgotten Fleet: The Mothball Navy', 0.39)
('World War II Sites in the United States: A Tour Guide & Directory', 0.38)



Recommendations for: Uniforms of World War II



Recommendations for: Reluctant Allies: German-Japanese Naval Relations in World War II
('Black Edelweiss: A Memoir of Combat and Conscience by a Soldier of the Waffen-SS', 0.36)



Recommendations for: The Yugoslav Auschwitz and the Vatican: The Croatian Massacre of the Serbs During World War II
('Concerning the Spiritual in Art', 0.25)
('Unholy Trinity : The Vatican, The Nazis, & Soviet Intelligence', 0.25)
('Witness to Jasenovacs Hell English/Serbian Edition', 0.18)
('Inat : Images of Serbia and the Kosovo Conflict', 0.0)



Recommendations for: S

Recommendations for: Blankets of Fire: U.S. Bombers over Japan During World War II (Smithsonian History of Aviation and Spaceflight (Paperback))
("The Last Mission: The Secret History of World War II's Final Battle", 0.62)
('To Command the Sky: The Battle for Air Superiority over Germany, 1942-1944 (Smithsonian History of Aviation and Spaceflight Series)', 0.6)
('Downfall : The End of the Imperial Japanese Empire', 0.45)
('The Dynamics of Military Revolution, 1300-2050', 0.3)



Recommendations for: The War: Stories of Life and Death from World War II
('Mob: Stories of Death and Betrayal from Organized Crime', 0.18)
('Deep Blue: Stories of Shipwreck, Sunken Treasure and Survival (Adrenaline Series)', 0.18)



Recommendations for: The Little Ships : The Heroic Rescue at Dunkirk in World War II
('In Flanders Field: The Story of the Poem', 0.2)



Recommendations for: Follow Me and Die: The Destruction of an American Division in World War II
('The Battle of Hurtgen Forest', 0.46)



Recom

...

Recommendations for: Into the Rising Sun: In Their Own Words, World War II's Pacific Veterans Reveal the Heart of Combat
("Beyond Valor: World War II's Ranger and Airborne Veterans Reveal the Heart of Combat", 0.9)
('Goodbye, Darkness: A Memoir of the Pacific War', 0.89)
("Beyond Valor: World War II's Ranger and Airborne Veterans Reveal the Heart of Combat", 0.8)
('With the Old Breed: At Peleliu and Okinawa', 0.5)
('Utmost Savagery', 0.4)
('Beyond Valor: World War II Ranger and Airborne Veterans Reveal the Heart of Combat', 0.31)
("Beyond Valor: World War II's Rangers and Airborne Veterans Reveal the Heart of Combat", 0.31)
('On Seas of Glory : Heroic Men, Great Ships, and Epic Battles of the American Navy', 0.23)



Recommendations for: Combat Medic: World War II



Recommendations for: Ernie Pyle's War: America's Eyewitness to World War II (Modern War Studies)
('Brave Men', 0.52)
('The Half Not Told: The Civil War in a Frontier Town', 0.19)
("Bill Mauldin's Army : Bill Mauldin's Grea

Recommendations for: V Was for Victory: Politics and American Culture During World War II
('The Hungry Years: A Narrative History of the Great Depression in America', 0.69)
("The Damned and the Beautiful: American Youth in the 1920's (Galaxy Books)", 0.6)
('Franklin D Roosevelt And The New Deal', 0.5)
('The War in American Culture : Society and Consciousness during World War II', 0.47)
('Reading the Holocaust (Canto)', 0.33)
('The Ordeal of Total War: 1939-1945', 0.25)
('Reading the Holocaust', 0.12)
('Days of Sadness Years of Triumph: The American People,1939-1945', 0.0)



Recommendations for: Cassell Military Classics: Storming Eagles: German Airborne Forces in World War II
('Fallschirmjagers in Crete', 0.67)
('Hunters from the Sky', 0.67)
("7th Flieger Division: Student's Fallschirmjager Elite (Spearhead Series)", 0.42)
('German Paratroops, Uniforms, Insignia & Equipment of the Fallschirmjager in Wwii: Uniforms, Insignia & Equipment of the Fallschirmjager in World War II (Schiffer 

Recommendations for: How to Model World War II German Armor
('Modeling Tanks and Military Vehicles', 1.0)
('German Tanks of World War II in Color (Enthusiast Color Series)', 0.15)



Recommendations for: How Hitler Could Have Won World War II : The Fatal Errors That Lead to Nazi Defeat
('Third Reich Victorious: The Alternate Decisions of World War II', 0.91)
('The Hitler Options: Alternate Decisions of World War II (Greenhill Military Paperbacks.)', 0.91)
('How Wars Are Won : The 13 Rules of War - from Ancient Greece to the War on Terror', 0.5)
('How Great Generals Win', 0.45)



Recommendations for: The Nisei Soldier : Historical Essays on World War II and the Korean War, 2nd ed.
('Unlikely Liberators: The Men of the 100th and 442nd', 0.53)



Recommendations for: Splinter Fleet: The Wooden Subchasers of World War II
('PC Patrol Craft of World War II', 0.9)



Recommendations for: Secret Soldiers: The Story of World War II's Heroic Army of Deception
('Ghost Army of World War II', 0.83

Recommendations for: Working-Class New York: Life and Labor Since World War II
('The Cio 1935-1955', 0.5)
('Gotham: A History of New York City to 1898 (The History of NYC Series)', 0.35)
('The War Against the Poor: The Underclass and Antipoverty Policy', 0.3)
('No Shame in My Game : The Working Poor in the Inner City', 0.25)
('Economic Apartheid in America: A Primer on Economic Inequality and Security', 0.17)



Recommendations for: The Originals: The Women's Auxiliary Ferrying Squadron of World War II
('Women Pilots Of World War II', 0.73)
('Yankee Doodle Gals : Women Pilots Of World War Ii', 0.48)
('A Wasp Among Eagles: A Woman Military Test Pilot in World War II', 0.45)
('Those Wonderful Women in Their Flying Machines: The Unknown Heroines of World War Two', 0.06)



Recommendations for: Endgame: The Betrayal and Fall of Srebrenica, Europe's Worst Massacre Since World War II
('Love Thy Neighbor : A Story of War', 0.83)
('Slaughterhouse: Bosnia and the Failure of the West', 0.73)
('S

Recommendations for: German Flak in World War II 1939-1945 (Schiffer Military/Aviation History)
('88 Mm Flak 18/36/37/41 & Pak 43 1936-45 (New Vanguard 46)', 0.5)
('German 20Mm Flak in World War II: 1935-1945 (Schiffer Military/Aviation History)', 0.0)



Recommendations for: The Divine Wind: Japan's Kamikaze Force in World War II
("Kamikaze: Japan's Suicide Gods", 0.36)
('Kamikaze, Cherry Blossoms, and Nationalisms : The Militarization of Aesthetics in Japanese History', 0.35)
('The Kamikazes : Suicide Squadrons of World War II', 0.35)
('Requiem for Battleship Yamato (Bluejacket Books)', 0.32)
('No Surrender: My Thirty-Year War (Bluejacket Books)', 0.27)



Recommendations for: The US Army of World War II, Volume 1: The Pacific (Men-At-Arms Series, 342)
('U.S. Paratrooper 1941-1945: Weapons, Armor, Tactics (Warrior Series, 26)', 1.0)
('The US Army in World War II, Volume 3: North-West Europe (Men-At-Arms Series, 350)', 0.83)
('British Army 1939-45: (1) North-West Europe (Men-at-Arms S

## Summary

Well done! In this lab, you effectively created a recommendation system for a real world dataset!